### Sentiment Analysis using LSTM

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

####  Prepare data

In [5]:
# Sentences (textual data) and their sentiment labels (1 for positive, 0 for negative)
sentences = ["i love this movie", "this film is amazing", "i didn't like it", "it was terrible"]
sentiment = [1, 1, 0, 0]

#### Create Vocabulary

In [6]:
# Simple vocabulary to represent words as indices
vocab = {"<PAD>": 0, "i": 1, "love": 2, "this": 3, "movie": 4, "film": 5, "is": 6, "amazing": 7, "didn't": 8, "like": 9, "it": 10, "was": 11, "terrible": 12}

We create a simple vocabulary to represent words as indices. This allows us to convert words in our sentences to numbers, which can be fed as input to our neural network.

#### Tokenize, encode and pad sentences

In [7]:
encoded_sentences = [[vocab[word] for word in sentence.split()] for sentence in sentences]
max_length = max([len(sentence) for sentence in encoded_sentences])
padded_sentences = [sentence + [vocab["<PAD>"]] * (max_length - len(sentence)) for sentence in encoded_sentences]

We tokenize and encode the sentences using the vocabulary created earlier. We also pad the sentences with the `<PAD>` token to make them all the same length.

#### Convert data to tensors

In [8]:
inputs = torch.LongTensor(padded_sentences)
labels = torch.FloatTensor(sentiment)

We convert the input data and labels to PyTorch tensors. Inputs are converted to LongTensors, while labels are converted to FloatTensors.

#### Define LSTM Model

In [9]:
class SimpleLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(SimpleLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, _) = self.lstm(embedded)
        logits = self.fc(hidden.squeeze(0))
        return logits


We define a simple LSTM model class that inherits from `nn.Module`. The model consists of an embedding layer, an LSTM layer, and a fully connected (linear) layer. The forward method takes an input tensor `x`, passes it through the embedding layer, the LSTM layer, and finally the fully connected layer to produce the output logits.

#### Instantiate model and define loss and optimizer

In [10]:
model = SimpleLSTM(len(vocab), embedding_dim=10, hidden_dim=20, output_dim=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

We instantiate the LSTM model with the vocabulary size, embedding dimensions, hidden dimensions, and output dimensions. We also define the binary cross-entropy with logits loss (`BCEWithLogitsLoss`) and the Adam optimizer.

#### Train the model

In [11]:
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()
    predictions = model(inputs.t()).squeeze(1)
    loss = criterion(predictions, labels)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")


Epoch: 100, Loss: 0.12670296430587769
Epoch: 200, Loss: 0.017672989517450333
Epoch: 300, Loss: 0.007727793417870998
Epoch: 400, Loss: 0.00463694054633379
Epoch: 500, Loss: 0.003190726973116398
Epoch: 600, Loss: 0.002372029237449169
Epoch: 700, Loss: 0.0018512976821511984
Epoch: 800, Loss: 0.0014917112421244383
Epoch: 900, Loss: 0.0012283801333978772
Epoch: 1000, Loss: 0.0010285137686878443


We train the model for 1000 epochs. In each epoch, we:

- Reset the gradients by calling optimizer.zero_grad()
- Get the model's predictions for the input sentences by calling model(inputs.t()).squeeze(1)
- Calculate the loss between the predictions and the true labels using the criterion defined earlier
- Perform backpropagation by calling loss.backward()
- Update the model's parameters by calling optimizer.step()
- We also print the loss every 100 epochs for monitoring the training progress.

#### Test the model

In [12]:
with torch.no_grad():
    test_sentences = ["i love this film", "it was terrible"]
    encoded_test_sentences = [[vocab[word] for word in sentence.split()] for sentence in test_sentences]
    padded_test_sentences = [sentence + [vocab["<PAD>"]] * (max_length - len(sentence)) for sentence in encoded_test_sentences]
    test_inputs = torch.LongTensor(padded_test_sentences)
    test_predictions = torch.sigmoid(model(test_inputs.t()).squeeze(1))
    print("Test predictions:", test_predictions)


Test predictions: tensor([9.9716e-01, 9.5173e-04])


We test the model on two new sentences. First, we tokenize, encode, and pad the test sentences in the same way as we did for the training sentences. We then convert the test sentences to PyTorch tensors and pass them through the model. We apply the sigmoid function to the output logits to obtain the final predictions, which represent the probability of each sentence being positive.

The resulting `test_predictions` tensor contains the model's sentiment predictions for the given test sentences.